---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
# df.head(10)

In [2]:
def reformat(d):
    if len(d) > 0:
        d = d.reset_index()
        d.rename(columns={'level_0':'ind'}, inplace=True)
#         print(d.match.value_counts(dropna=False))
    return d

In [3]:
def date_sorter():
    
    df_ans = df.str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})')
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    without_day = df[not_duplicated].str.extractall('(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4})')
    without_day = without_day.append(df[not_duplicated].str.extractall(r'(?P<month>\d\d?)/(?P<year>\d{4})'))
    without_day['day'] = 1
    df_ans = df_ans.append(without_day)
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    without_month = df[not_duplicated].str.extractall(r'(?P<year>\d{4})')
    without_month['day'] = 1
    without_month['month'] = 1
    df_ans = df_ans.append(without_month)
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    # Year
    df_ans['year'] = df_ans['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    df_ans['year'] = df_ans['year'].apply(lambda x: str(x))

    # Month
    df_ans['month'] = df_ans['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = {'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1,'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3, 'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8}
    
    df_ans = df_ans.replace(month_dict)
    df_ans['month'] = df_ans['month'].apply(lambda x: str(x))
    df_ans['day'] = df_ans['day'].apply(lambda x: str(x))

    df_ans['date'] = df_ans['month'] + '/' + df_ans['day'] + '/' + df_ans['year']
    df_ans['date'] = pd.to_datetime(df_ans['date'])

    df_ans = df_ans.sort_values(by='date')
    return_rank = pd.Series(list(df_ans.index.labels[0]))
    
    return return_rank # Your answer here

In [4]:
# import re
# import datetime

# df1 = df.str.extractall(r'(?P<month>\d{1,2})[/|-](?P<day>\d{1,2})[/|-](?P<year>(?:\d{4}|\d{2}))\b') #good as is
# df2 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)-?/?.? ?(?P<day>\d{1,2})-?/?.?,? ?(?P<year>\d{4})\b')
# df3 = df.str.extractall(r'(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),?.? ?(?P<year>\d{4})\b')
# df4 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (?P<day>\d{1,2}[a-zA-Z]{2}), (?P<year>\d{4})\b')
# df5 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (?P<year>\d{4})\b') 
# df6 = df.str.extractall(r'(?P<month>[0,1]?\d)/(?P<year>\d{4})\b') #good as is
# df7 = df.str.extractall(r'(?P<year>[12]\d{3})\b') #if not in 
# #     print('df1: ', len(df1))
# #     print('df2: ', len(df2))
# #     print('df3: ', len(df3))
# #     print('df4: ', len(df4))
# #     print('df5: ', len(df5))
# #     print('df6: ', len(df6))
# #     print('df7: ', len(df7))
# #     print(len(df) - len(df1) - len(df2) - len(df3) - len(df4) - len(df5) - len(df6) - len(df7))

# dfs = [df1,df2,df3,df4,df5,df6,df7]

# df1 = reformat(df1)
# df2 = reformat(df2)
# df3 = reformat(df3)
# df4 = reformat(df4)
# df5 = reformat(df5)
# df6 = reformat(df6)
# df7 = reformat(df7)

# fix5 = set(df5.ind.values) - set(df3.ind.values)
# df5 = df5[df5.ind.isin(fix5)]
# len(fix5), len(df5)

# fix6 = set(df6.ind.values) - set(df1.ind.values)
# df6 = df6[df6.ind.isin(fix6)]
# len(fix6), len(df6)

# fix7 = set(df7.ind.values) - set(df1.ind.values) - set(df2.ind.values) - set(df3.ind.values) - \
# set(df5.ind.values) - set(df6.ind.values)
# df7 = df7[df7.ind.isin(fix7)]
# len(fix7), len(df7)

# monthdict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}

# df1['month'] = df1.month.astype(int)
# df1['day'] = df1.day.astype(int)
# df1['year'] = df1.year.astype(int)
# df1['year'] = [int('19' + str(x)) if len(str(x))==2 else int(x) for x in df1.year]
# df1['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df1.month,df1.day,df1.year)]
# df1['date'] = pd.to_datetime(df1.date)
# df1 = df1[['ind','date']]

# df2['newmon'] = [monthdict[x[:3]] for x in df2.month]
# df2['day'] = df2.day.astype(int)
# df2['year'] = df2.year.astype(int)
# df2['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df2.month,df2.day,df2.year)]
# df2['date'] = pd.to_datetime(df2.date)
# df2 = df2[['ind','date']]

# df3['month'] = [monthdict[x[:3]] for x in df3.month]
# df3['day'] = df3.day.astype(int)
# df3['year'] = df3.year.astype(int)
# df3['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df3.month,df3.day,df3.year)]
# df3['date'] = pd.to_datetime(df3.date)
# df3 = df3[['ind','date']]

# df5['month'] = [monthdict[x[:3]] for x in df5.month]
# df5['day'] = 1
# df5['day'] = df5.day.astype(int)
# df5['year'] = df5.year.astype(int)
# df5['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df5.month,df5.day,df5.year)]
# df5['date'] = pd.to_datetime(df5.date)
# df5 = df5[['ind','date']]

# df6['month'] = df6.month.astype(int)
# df6['day'] = 1
# df6['year'] = df6.year.astype(int)
# df6['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df6.month,df6.day,df6.year)]
# df6['date'] = pd.to_datetime(df6.date)
# df6 = df6[['ind','date']]

# df7['month'] = 1
# df7['day'] = 1
# df7['year'] = df7.year.astype(int)
# df7['date'] = [f'{x}/{y}/{z}' for (x,y,z) in zip(df7.month,df7.day,df7.year)]
# df7['date'] = pd.to_datetime(df7.date)
# df7 = df7[['ind','date']]

# alldf = pd.concat([df1,df2,df3,df5,df6,df7])
# alldf.sort_values(by='date', inplace=True)
# alldf.reset_index(drop=True,inplace=True)

# ser = pd.Series(alldf.ind)

# ser

In [5]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64